# __Hyperparameter Tuning__
- Hyperparameter tuning is the process of systematically searching for the best combination of hyperparameter values for a machine learning model.
- It involves selecting a subset of hyperparameters and exploring different values for each hyperparameter to find the configuration that optimizes the model's performance on a given dataset.

Let's understand how it works.

## Steps to be followed:
1. Import the required libraries
2. Load the dataset and standardize it
3. Build and Train a Basic Deep Learning Model
4. Define the HyperModel
5. Instantiate the Tuner and Perform Hyperparameter Tuning
6. Evaluate Both Models Using Multiple Metrics

### Step 1: Import the required libraries


In [1]:
!pip install tensorflow==2.17.0 scikeras==0.13.0 keras==3.2.0

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install keras-tuner

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.0 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.3 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import os

# Disable oneDNN optimizations to avoid potential minor numerical differences caused by floating-point round-off errors.
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

In [4]:
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import optimizers

import keras_tuner
from keras_tuner import HyperModel
from keras_tuner.tuners import RandomSearch
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

2024-11-06 09:51:42.991612: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-06 09:51:43.006384: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-06 09:51:43.010815: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-06 09:51:43.022570: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-06 09:51:45.547752: W ten

### Step 2: Load the dataset and standardize it
- In this code, the breast cancer dataset is loaded using the **load_breast_cancer** function.
- The features are stored in the **X** variable, and the corresponding labels are stored in **y**.
- Split the dataset into training and testing sets, then standardizes the training data to have zero mean and unit variance, applying the same transformation to the test data.

In [5]:
# Load the Breast Cancer dataset
data = load_breast_cancer()
X = data.data
y = data.target

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
train_sc = scaler.fit_transform(X_train)
test_sc = scaler.transform(X_test)

### Step 3: Build and Train a Basic Deep Learning Model

- Constructs a sequential neural network with two hidden layers of 32 and 16 neurons respectively, using ReLU activation, and a dropout layer to reduce overfitting. The output layer uses a sigmoid activation function for binary classification.

- Prepares the model for training by specifying the Adam optimizer, binary cross-entropy loss function for binary classification, and tracks the accuracy metric.

- Fits the model on the standardized training data for 100 epochs, using 10% of it as a validation set to monitor performance, without verbosity to minimize output during training.

- Assesses the model's performance on the standardized test data, obtaining the loss and accuracy, then prints the accuracy to give an indication of how well the model predicts unseen data.

In [6]:
# Build a basic model
basic_model = Sequential([
    Dense(32, activation='relu', input_shape=(train_sc.shape[1],)),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
basic_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
basic_model.fit(train_sc, y_train, epochs=100, validation_split=0.1, verbose=0)

# Evaluate the model on the test set
basic_loss, basic_accuracy = basic_model.evaluate(test_sc, y_test)
print("Basic Model Accuracy: ", basic_accuracy)

/voc/work/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-11-06 09:51:55.584137: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9738 - loss: 0.1061 
Basic Model Accuracy:  0.9736841917037964


### Step 4:  Define the HyperModel
Defines a custom class MyHyperModel that extends the HyperModel class from Keras Tuner, used for hyperparameter tuning.

- The class `MyHyperModel` is designed to construct a neural network model dynamically, with varying hyperparameters.
- The `__init__` method initializes the class with an `input_shape`, which is the shape (number of features) of the input data that the model will expect. This is stored as a class attribute to be used later in building the model.

- The build method creates a neural network model architecture with tunable hyperparameters.
- `hp.Int('units', min_value=10, max_value=100, step=10)` This line specifies that the number of units in the Dense layers should be treated as a hyperparameter, with possible values ranging from 10 to 100 in steps of 10.

- `hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)` This specifies that the dropout rate should also be a hyperparameter, ranging from 0.0 to 0.5 with a step of 0.1.

- `model.add(Dense(1, activation='sigmoid'))` Adds an output layer with a single unit and sigmoid activation suitable for binary classification.

- Learning rate for the Adam optimizer is configured with another tunable parameter `(hp.Float('learning_rate', ...))` which varies logarithmically from 0.0001 to 0.01.


In [7]:
class MyHyperModel(HyperModel):
    def __init__(self, input_shape):
        self.input_shape = input_shape

    def build(self, hp):
        model = Sequential()
        model.add(Dense(
            units=hp.Int('units', min_value=10, max_value=100, step=10),
            activation='relu', input_shape=(self.input_shape,)
        ))
        model.add(Dropout(
            hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)
        ))
        model.add(Dense(
            units=hp.Int('units', min_value=10, max_value=100, step=10),
            activation='relu'
        ))
        model.add(Dense(1, activation='sigmoid'))
        model.compile(
            optimizer=tf.keras.optimizers.Adam(
                hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        return model

### Step 5: Instantiate the Tuner and Perform Hyperparameter Tuning
- Conducts hyperparameter tuning using Keras Tuner's RandomSearch, optimizing the neural network's configuration to maximize validation accuracy by testing different combinations of model parameters and identifying the best performing model.

- It sets up a hyperparameter optimization process targeting the validation accuracy for a model defined by hypermodel.

- The process will try up to 10 different sets of hyperparameters, running each configuration twice to ensure stability in the reported performance metrics, all within the specified project directory for organized storage and potential review.

- This approach is useful for exploring a potentially vast hyperparameter space more efficiently than exhaustively testing all combinations.

In [ ]:
# Assuming 'train_sc' and 'y_train' are defined as your scaled training data and labels
input_shape = train_sc.shape[1]  # Extract the number of features

# Create an instance of the HyperModel
hypermodel = MyHyperModel(input_shape=input_shape)

# Instantiate the tuner
tuner = RandomSearch(
    hypermodel,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=2,
    directory='tuner_data',
    project_name='breast_cancer_optimization'
)

# Perform hyperparameter tuning
tuner.search(train_sc, y_train, epochs=50, validation_split=0.2)

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

Trial 8 Complete [00h 00m 06s]
val_accuracy: 0.9835164844989777

Best val_accuracy So Far: 0.9835164844989777
Total elapsed time: 00h 00m 52s

Search: Running Trial #9

Value             |Best Value So Far |Hyperparameter
20                |80                |units
0.1               |0.3               |dropout
0.00014531        |0.0080711         |learning_rate

Epoch 1/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.4371 - loss: 0.7427 - val_accuracy: 0.5824 - val_loss: 0.6933
Epoch 2/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4580 - loss: 0.7555 - val_accuracy: 0.5934 - val_loss: 0.6733
Epoch 3/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5775 - loss: 0.7112 - val_accuracy: 0.6484 - val_loss: 0.6544
Epoch 4/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6731 - loss: 0.6838 - val_accuracy: 0.6703 - val_loss: 0.6373
Epoch 5/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6799 - loss: 0.6733 - val_accuracy: 0.6923 - val_loss: 0.6203
Epoc

#### **Observation:**

- Trial 10 Complete: Indicates the completion of the 10th trial in the hyperparameter tuning process.

- val_accuracy: Indicates the validation accuracy achieved by the model configuration tested in the 10th trial. It indicates how well the model performed on the validation set, which is a subset of the training data not used for training the model but for evaluating its performance.

- Best val_accuracy So Far: 0.9890109896659851: Up to this point in the tuning process, this is the highest validation accuracy achieved across all trials. This suggests that the model configuration from the 10th trial is currently the best performer among all configurations tested.

### Step 6: : Evaluate Both Models Using Multiple Metrics

In [ ]:
# Predict probabilities for the basic and best models
basic_predictions_proba = basic_model.predict(test_sc)
basic_predictions = (basic_predictions_proba > 0.5).astype(int)

best_predictions_proba = best_model.predict(test_sc)
best_predictions = (best_predictions_proba > 0.5).astype(int)

# Print classification report for basic model
print("Basic Model Classification Report:")
print(classification_report(y_test, basic_predictions, target_names=['Benign', 'Malignant']))

# Calculate and print ROC AUC for the basic model
basic_auc = roc_auc_score(y_test, basic_predictions_proba)
print("Basic Model ROC AUC:", basic_auc)

# Print classification report for best model
print("Best Model Classification Report:")
print(classification_report(y_test, best_predictions, target_names=['Benign', 'Malignant']))

# Calculate and print ROC AUC for the best model
best_auc = roc_auc_score(y_test, best_predictions_proba)
print("Best Model ROC AUC:", best_auc)


#### **Observation:**

**Basic Model Results:**

- Classification Report:
    - Benign: Precision of 0.98, recall of 0.95, and F1-score of 0.96. High precision and recall suggest the model effectively identifies benign cases.
    - Malignant: Precision of 0.97, recall of 1, and F1-score of 0.98. This shows excellent performance in identifying malignant cases with very high precision.
    - Overall Accuracy: 97%, indicating a high overall rate of correct predictions.
- ROC AUC: 0.99, which is very close to 1. This score indicates an excellent ability to discriminate between the benign and malignant classes.

**Best Model Results:**

- Classification Report:
    - Benign: Perfect precision of 1.00 and recall of 0.95, resulting in an F1-score of 0.98.
    - Malignant: Precision of 0.97 and a perfect recall of 1.00, with an F1-score of 0.99. This configuration slightly improves in identifying all malignant cases compared to the basic model.
    - Overall Accuracy: 98%, a slight improvement over the basic model.
- ROC AUC: 0.99, indicating a marginal but notable improvement in discriminative ability over the basic model.

As per the results, the best model, which underwent hyperparameter tuning, shows improved performance metrics across most areas compared to the basic model. While both models perform exceptionally well, the best model's perfect recall for malignant cases and higher overall ROC AUC suggest it is slightly more reliable, particularly in scenarios where failing to detect malignant cases (false negatives) is critically risky